In [122]:
%%sql
postgresql:///groceries
    -- ap represents all products purchased in 2020 --
WITH ap AS (SELECT CAST(EXTRACT(MONTH FROM (TO_DATE(fulldate, 'YYYY-MM-DD'))) AS INTEGER) AS month, 
             COUNT(p20.purchaseid) AS total_sales_1 
             FROM purchases_2020 AS p20
             GROUP BY month),

--dp20 represents dairy products purchased in 2020 --
dp20 AS (SELECT CAST(EXTRACT(MONTH FROM (TO_DATE(fulldate, 'YYYY-MM-DD'))) AS INTEGER) AS month, 
        COUNT(p20.purchaseid) AS total_sales 
        FROM purchases_2020 AS p20 
        LEFT JOIN categories AS c ON c.purchase_id = p20.purchaseid 
        WHERE category IN ('whole milk', 'yogurt', 'domestic eggs') 
        GROUP BY month), 

-- dp19 represents dairy products purchased in 2019 --
dp19 AS (SELECT month, COUNT(p19.purchase_id) AS total_sales_2 
        FROM purchases_2019 AS p19 
        LEFT JOIN categories AS c ON c.purchase_id = p19.purchase_id 
        WHERE category IN ('whole milk', 'yogurt', 'domestic eggs') 
        GROUP BY month)

SELECT CAST(dp20.month AS INTEGER), CAST(total_sales AS INTEGER),
ROUND(CAST(total_sales::FLOAT * 100/total_sales_1 AS NUMERIC), 2) AS market_share,
ROUND(CAST((total_sales - total_sales_2) * 100/total_sales_2::FLOAT AS NUMERIC), 2) AS year_change
FROM ap 
LEFT JOIN dp20 USING (month)
LEFT JOIN dp19 ON ap.month = dp19.month 
ORDER BY month

12 rows affected.


month,total_sales,market_share,year_change
1,228,12.47,48.05
2,165,11.11,5.10
3,216,12.05,27.81
4,218,13.09,26.01
5,195,11.40,17.47
6,196,10.94,20.99
7,195,11.85,-0.51
8,231,11.77,32.00
9,180,11.16,4.65
10,193,11.61,17.68
